In [3]:
#Jesús Matos Torres, Ricardo Cárdenes Pérez, Susana Suárez Mendoza, Jia Hao Yang, Óscar Rico Rodríguez y Aurora Zouris

import IPython
import soundfile as sf
import numpy as np
import io
import pandas as pd
from urllib.request import urlopen
import torch
import torchaudio
import matplotlib.pyplot as plt
import requests
from torchlibrosa.stft import Spectrogram
import librosa
import librosa.display 
import tensorflow as tf
import os
import wave
import pylab
from pathlib import Path
from scipy import signal
from scipy.io import wavfile
from sklearn.metrics import confusion_matrix
import itertools
from scipy.io import wavfile as wav

In [ ]:
def play_audio_fragment(filename, start, end, samplerate = 50000):
    """Play a fragment of an audio file.
    Args:
        filename: path to the audio file
        start: start of the fragment in samples
        end: end of the fragment in samples
        samplerate: samplerate to use when reading the file"""
    if not filename.startswith("."):
      filename = f"https://storage.googleapis.com/datathon2022/dataset1/{filename}.ogg"

    if filename.startswith("http"):
        filename = io.BytesIO(urlopen(filename).read())

    data, read_sr = sf.read(filename, start=start, stop=end)
    assert samplerate == read_sr, f"samplerate does not match {samplerate} (from file) != {read_sr} (function parameter)"

    IPython.display.display(IPython.display.Audio(data, rate=samplerate))

def play_annotation_from_df(row, margin: int = 0, samplerate = 50000):
    """Play a fragment of a wav file in a jupyter notebook.
    Args:
        row: a row of a pandas dataframe with the following columns:
            - path: path to the wav file
            - offset: offset in seconds
            - duration: duration in seconds
        margin: margin in seconds to add to the start and end of the fragment
        samplerate: samplerate to use when reading the file
        """
    m = margin * samplerate # margin in samples
    start = max(int(np.floor(row['start'] - m)), 0)
    end = int(np.ceil(row['start'] + row['duration'] * samplerate + m))
    filename = row['path']
    print(row)
    play_audio_fragment(filename, start, end, samplerate)

In [1]:
dataset1 = pd.read_csv("https://bit.ly/dataset1marine")
if not isinstance(dataset1, pd.DataFrame):
  assert False, "dataset1 is not a DataFrame"

print(dataset1)

NameError: name 'pd' is not defined

In [ ]:
spectrogram_extractor = Spectrogram(
    win_length=1024, 
    hop_length=320
).cuda()

In [ ]:
# get raw audio data
example, _ = librosa.load('input/first.wav', sr=32000, mono=True)
raw_audio = torch.Tensor(example).unsqueeze(0).cuda()
spectrogram = spectrogram_extractor(raw_audio)

In [ ]:
plt.figure(figsize=(25, 20))
librosa.display.specshow(spectrogram)

In [ ]:
# Set paths to input and output data
INPUT_DIR = '/content/input'
OUTPUT_DIR = '/content/output'

# Print names of 10 WAV files from the input path
parent_list = os.listdir(INPUT_DIR)
for i in range(len(parent_list)):
    print(parent_list[i])

signal_wave = spectrogram
sample_rate = 16000
sig = np.frombuffer(signal_wave.readframes(sample_rate), dtype=np.int16)
plt.figure(figsize=(12,12))
plot_a = plt.subplot(211)
plot_a.set_title(parent_list[i])
plot_a.plot(sig)
plot_a.set_xlabel('sample rate * time')
plot_a.set_ylabel('energy')

plot_b = plt.subplot(212)
plot_b.specgram(sig, NFFT=1024, Fs=sample_rate, noverlap=900)
plot_b.set_xlabel('Time')
plot_b.set_ylabel('Frequency')

plt.show()

In [ ]:
sample_rate, wave_audio = wav.read('input/first.wav')
wave_audio

In [ ]:
data, sample_rate = librosa.load('input/second.ogg')
plt.figure(figsize=(500,50))
librosa.display.waveshow(data, sr = sample_rate)

In [ ]:
for i in range(1): 
    signal_wave = data
    sample_rate = 16000

    plt.figure(figsize=(12,12))
    plot_a = plt.subplot(211)
    plot_a.set_title(parent_list[i])
    plot_a.plot(data)
    plot_a.set_xlabel('sample rate * time')
    plot_a.set_ylabel('energy')

    plot_b = plt.subplot(212)
    plot_b.specgram(data, NFFT=1024, Fs=sample_rate, noverlap=15000)
    plot_b.set_xlabel('Time')
    plot_b.set_ylabel('Frequency')

plt.show()

In [ ]:
data, sample_rate = librosa.load('/content/input/a5f4867089fe98f346a7759237f78323.ogg')
plt.figure(figsize=(500,50))
librosa.display.waveshow(data, sr = sample_rate)

for i in range(1): 
    
    sample_rate = 16000

    plt.figure(figsize=(12,12))
    plot_a = plt.subplot(211)
    plot_a.set_title(parent_list[i])
    plot_a.plot(data)
    plot_a.set_xlabel('sample rate * time')
    plot_a.set_ylabel('energy')

    plot_b = plt.subplot(212)
    plot_b.specgram(data, NFFT=1024, Fs=sample_rate, noverlap=900)
    plot_b.set_xlabel('Time')
    plot_b.set_ylabel('Frequency')

plt.show()

In [ ]:
parent_list = os.listdir(INPUT_DIR)
pf = []
for i in range(len(parent_list)):
  if (parent_list[i][-3] == 'o' or parent_list[i][-3] == 'w'):
    pf.append(parent_list[i])

parent_list = pf

print(parent_list)

for i in range(len(parent_list)): 
    
    sample_rate = 16000

    print(f'input/{parent_list[i]}')
    data, sample_rate = librosa.load(f'input/{parent_list[i]}')
    print(data)

    plt.figure(figsize=(12,12))
    plot_b = plt.subplot(212)
    plot_b.specgram(data, NFFT=1024, Fs=sample_rate, noverlap=900)
    plot_b.set_xlabel('Time')
    plot_b.set_ylabel('Frequency')

    plt.savefig(f'images2/example{i}.png')
    plt.show()

In [ ]:
# Utility function to get sound and frame rate info
def get_wav_info(wav_file):
    wav = wave.open(wav_file, 'r')
    frames = wav.readframes(-1)
    sound_info = pylab.frombuffer(frames, 'int16')
    frame_rate = wav.getframerate()
    wav.close()
    return sound_info, frame_rate

# For every recording, make a spectogram and save it as label_speaker_no.png
if not os.path.exists(os.path.join(OUTPUT_DIR, 'audio-images')):
    os.mkdir(os.path.join(OUTPUT_DIR, 'audio-images'))
    
for filename in os.listdir(INPUT_DIR):
    if "wav" in filename:
        file_path = os.path.join(INPUT_DIR, filename)
        file_stem = Path(file_path).stem
        target_dir = f'class_{file_stem[0]}'
        dist_dir = os.path.join(os.path.join(OUTPUT_DIR, 'audio-images'), target_dir)
        file_dist_path = os.path.join(dist_dir, file_stem)
        if not os.path.exists(file_dist_path + '.png'):
            if not os.path.exists(dist_dir):
                os.mkdir(dist_dir)
            file_stem = Path(file_path).stem
            sound_info, frame_rate = get_wav_info(file_path)
            pylab.specgram(sound_info, Fs=frame_rate)
            pylab.savefig(f'{file_dist_path}.png')
            pylab.close()

# Print the ten classes in our dataset
path_list = os.listdir(os.path.join(OUTPUT_DIR, 'audio-images'))
print("Classes: \n")
for i in range(10):
    print(path_list[i])
    
# File names for class 1
path_list = os.listdir(os.path.join(OUTPUT_DIR, 'audio-images/class_1'))
print("\nA few example files: \n")
for i in range(10):
    print(path_list[i])

In [ ]:
pf = []
parent_list = os.listdir(os.path.join('/content/', 'images'))

for i in range(len(parent_list)):
  if (parent_list[i][0] != '.'):
    pf.append(parent_list[i])

parent_list = pf
print(parent_list)

In [ ]:
# Declare constants
IMAGE_HEIGHT = 256
IMAGE_WIDTH = 256
BATCH_SIZE = 32
N_CHANNELS = 3
N_CLASSES = 2

# Make a dataset containing the training spectrograms
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
                                             batch_size=BATCH_SIZE,
                                             validation_split=0.2,
                                             directory=os.listdir(os.path.join('/content/images/', 'class_0')),
                                             shuffle=True,
                                             color_mode='rgb',
                                             image_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
                                             subset="training",
                                             seed=0)

# Make a dataset containing the validation spectrogram
valid_dataset = tf.keras.preprocessing.image_dataset_from_directory(
                                             batch_size=BATCH_SIZE,
                                             validation_split=0.2,
                                             directory=os.listdir(os.path.join('/content/images', 'images')),
                                             shuffle=True,
                                             color_mode='rgb',
                                             image_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
                                             subset="validation",
                                             seed=0)